# Segmentation and clustering assignment

## Getting data

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# get the respone html for url 
linkurl='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html=requests.get(linkurl).text

In [3]:
# parse the response html using beautifulsoup xml parser
soup=BeautifulSoup(html,'lxml')

#find the table and load the relevant data
tabl=soup.find('table')

### Build the initial pandas dataframe

In [4]:
#Build a pandas dataframe 
data=[]
headers=[]

#parse through the table tags and obtain data and header values
for rowtext in tabl.find_all('tr'):
    if(headers==[]):
        columns =rowtext.find_all('th')
        if columns!=[]: 
            headers=[columns[0].text,columns[1].text,columns[2].text]
    else:
        columns =rowtext.find_all('td')
        
        if columns!=[]: 
            #build row values
            row=[columns[0].text.strip('\n'),columns[1].text.strip('\n'),columns[2].text.strip('\n')]
            data.append(row)
            
#Column names as given in the assignment
headers=['PostalCode', 'Borough', 'Neighborhood']

#Initial dataframe
df=pd.DataFrame(data,columns=headers)  
df.shape

(287, 3)

## Cleaning Data

### Removing rows wth unassigned Borough

In [5]:
#If Borough value is Not assigned remove that row
assigned=df['Borough']!= 'Not assigned'
dfclean=df[assigned]
dfclean.shape

(210, 3)

### Replacing the unassigned neighborhood values with Borough value

In [6]:
for i in range(0,dfclean.shape[0]):
    if(dfclean.iloc[i]['Neighborhood']=='Not assigned'):
               dfclean.iloc[i]['Neighborhood']=dfclean.iloc[i]['Borough']
dfclean.shape

(210, 3)

### Merging the rows with same postalcode but different neoghborhood 

In [7]:
dfuniq=dfclean.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
dfuniq.shape

(103, 3)

## Final result

In [8]:
dfuniq.shape

(103, 3)

### Saving the dataframe

In [9]:
dfuniq.to_csv('Week3Part1.csv',header=True,index=False)